In [594]:
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as st
import math

In [595]:
select = st.norm.rvs(size=1000, loc=3, scale=2)
n = len(select)
df = pd.DataFrame(select)
mx = df.mean()[0]
dx = df.var()[0]**(1/2)
alpha = 0.05

In [596]:
k = 1 + int(1.78 * (len(select))**(1/3))

In [597]:
k

18

In [598]:
intervals = np.arange(min(select), max(select), abs(min(select) - max(select)) / k)

In [599]:
intervals

array([-5.83458802, -5.01783839, -4.20108877, -3.38433915, -2.56758952,
       -1.7508399 , -0.93409028, -0.11734066,  0.69940897,  1.51615859,
        2.33290821,  3.14965784,  3.96640746,  4.78315708,  5.5999067 ,
        6.41665633,  7.23340595,  8.05015557])

In [600]:
min(intervals), max(intervals)

(-5.834588015458953, 8.050155572215964)

In [601]:
p = [st.norm.cdf(intervals[0], loc=mx, scale=dx) - st.norm.cdf(min(select), loc=mx, scale=dx)]
for i in range(1, len(intervals)):
    p.append(st.norm.cdf(intervals[i], loc=mx, scale=dx) - st.norm.cdf(intervals[i - 1], loc=mx, scale=dx))

In [602]:
p

[0.0,
 2.8655206076828158e-05,
 0.00013756134186870996,
 0.0005640898446464674,
 0.0019759539402620457,
 0.005912868284704796,
 0.015115656645150456,
 0.033012320728668956,
 0.06159652328984657,
 0.09819204262322817,
 0.13373437224200807,
 0.15561839499524577,
 0.1547149113105959,
 0.13141855296933702,
 0.09537445603903172,
 0.059136239022465786,
 0.03132670342005228,
 0.014177709063306176]

In [603]:
per = {i: 0 for i in range(len(intervals))}
for x in select:
    for i in range(len(intervals)):
        if (x < intervals[i]) or ((x - intervals[i]) < 10**-10):
            per[i] += 1
            break

In [604]:
per

{0: 1,
 1: 0,
 2: 1,
 3: 0,
 4: 1,
 5: 3,
 6: 16,
 7: 37,
 8: 65,
 9: 96,
 10: 133,
 11: 144,
 12: 148,
 13: 135,
 14: 103,
 15: 66,
 16: 36,
 17: 12}

In [605]:
len(intervals)

18

In [606]:
for i in range(len(intervals)):
    if n * p[i] < 5:
        if i == 0:
            intervals[i] = 0
        elif i + 1 == len(intervals):
            intervals[i - 1] = 0
        else:
            min_per = list(per.values()).index(min(per[i - 1], per[i + 1]))
            if min_per < i:
                intervals[i - 1] = 0
            else:
                intervals[i] = 0

In [607]:
intervals

array([ 0.        ,  0.        ,  0.        ,  0.        , -2.56758952,
       -1.7508399 , -0.93409028, -0.11734066,  0.69940897,  1.51615859,
        2.33290821,  3.14965784,  3.96640746,  4.78315708,  5.5999067 ,
        6.41665633,  7.23340595,  8.05015557])

In [608]:
intervals = np.delete(intervals, np.where(intervals == 0))
intervals

array([-2.56758952, -1.7508399 , -0.93409028, -0.11734066,  0.69940897,
        1.51615859,  2.33290821,  3.14965784,  3.96640746,  4.78315708,
        5.5999067 ,  6.41665633,  7.23340595,  8.05015557])

In [609]:
per = {i + 1: 0 for i in range(len(intervals))}
for x in select:
    for i in range(len(intervals)):
        if (x < intervals[i]) or ((x - intervals[i]) < 10**-10):
            per[i + 1] += 1
            break

In [610]:
per

{1: 3,
 2: 3,
 3: 16,
 4: 37,
 5: 65,
 6: 96,
 7: 133,
 8: 144,
 9: 148,
 10: 135,
 11: 103,
 12: 66,
 13: 36,
 14: 12}

In [611]:
p = [st.norm.cdf(intervals[0], loc=mx, scale=dx) - st.norm.cdf(min(select), loc=mx, scale=dx)]
for i in range(1, len(intervals)):
    p.append(st.norm.cdf(intervals[i], loc=mx, scale=dx) - st.norm.cdf(intervals[i - 1], loc=mx, scale=dx))

In [612]:
p

[0.002706260332854051,
 0.005912868284704796,
 0.015115656645150456,
 0.033012320728668956,
 0.06159652328984657,
 0.09819204262322817,
 0.13373437224200807,
 0.15561839499524577,
 0.1547149113105959,
 0.13141855296933702,
 0.09537445603903172,
 0.059136239022465786,
 0.03132670342005228,
 0.014177709063306176]

In [613]:
s = 0
for i in range(len(per)):
    if p[i] == 0:
        pz = 10**-6
        s += ((per[i + 1] - n*pz)**2) / (n * pz)
    else:
        s += ((per[i + 1] - n*p[i])**2) / (n * p[i])

In [614]:
s

5.935774102087979

In [615]:
chi2_alpha = st.chi2.ppf(1 - alpha, len(p) - 2 - 1)

In [616]:
chi2_alpha

19.67513757268249

In [617]:
s < chi2_alpha

True

In [637]:
select = st.norm.rvs(size=1000, loc=3, scale=2)
n = len(select)
df = pd.DataFrame(select)
mx = df.mean()[0]
dx = (3 * df.var()[0])**(1/2) / math.pi
alpha = 0.05

In [638]:
k = 1 + int(1.78 * (len(select))**(1/3))

In [639]:
intervals = np.arange(min(select), max(select), abs(min(select) - max(select)) / k)

In [640]:
p = [st.logistic.cdf(intervals[0], loc=mx, scale=dx) - st.logistic.cdf(min(select), loc=mx, scale=dx)]
for i in range(1, len(intervals)):
    p.append(st.logistic.cdf(intervals[i], loc=mx, scale=dx) - st.logistic.cdf(intervals[i - 1], loc=mx, scale=dx))

In [641]:
per = {i: 0 for i in range(len(intervals))}
for x in select:
    for i in range(len(intervals)):
        if (x < intervals[i]) or ((x - intervals[i]) < 10**-10):
            per[i] += 1
            break

In [642]:
for i in range(len(intervals)):
    if n * p[i] < 5:
        if i == 0:
            intervals[i] = 0
        elif i + 1 == len(intervals):
            intervals[i - 1] = 0
        else:
            min_per = list(per.values()).index(min(per[i - 1], per[i + 1]))
            if min_per < i:
                intervals[i - 1] = 0
            else:
                intervals[i] = 0

In [643]:
intervals = np.delete(intervals, np.where(intervals == 0))
intervals

array([-2.54861664, -1.85059305, -1.15256946, -0.45454587,  0.24347772,
        0.94150132,  1.63952491,  2.3375485 ,  3.03557209,  3.73359569,
        4.43161928,  5.12964287,  5.82766646,  6.52569006,  7.22371365,
        7.92173724,  8.61976083])

In [644]:
per = {i + 1: 0 for i in range(len(intervals))}
for x in select:
    for i in range(len(intervals)):
        if (x < intervals[i]) or ((x - intervals[i]) < 10**-10):
            per[i + 1] += 1
            break

In [645]:
p = [st.logistic.cdf(intervals[0], loc=mx, scale=dx) - st.logistic.cdf(min(select), loc=mx, scale=dx)]
for i in range(1, len(intervals)):
    p.append(st.logistic.cdf(intervals[i], loc=mx, scale=dx) - st.logistic.cdf(intervals[i - 1], loc=mx, scale=dx))

In [646]:
s = 0
for i in range(len(per)):
    if p[i] == 0:
        pz = 10**-6
        s += ((per[i + 1] - n*pz)**2) / (n * pz)
    else:
        s += ((per[i + 1] - n*p[i])**2) / (n * p[i])

In [647]:
chi2_alpha = st.chi2.ppf(1 - alpha, len(p) - 2 - 1)

In [648]:
s < chi2_alpha

False

In [649]:
chi2_alpha

23.684791304840576

In [650]:
s

31.950492419323957